In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import openai
from dotenv import load_dotenv

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs

from method.ours.utils import create_driver, get_xpath
from method.ours.parse import parse_form
from method.ours.prompts import get_form_context
from method.ours.history import HistoryTable
from method.ours.generation import (
    generate_constraints_for_input_groups,
    generate_value_for_input_group,
    generate_values_for_input_groups,
    fill_form_with_value_table,
    submit_form
)
from method.ours.feedback import (
    get_local_feedback,
    get_global_feedback
)

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Global Variables
HEADLESS = False
TEXT_EMBEDDING_METHOD = 'ADA' # ['ADA', 'WORD2VEC', 'SPACY']
GRAPH_EMBEDDING_METHOD = 'NODE2VEC' # ['NODE2VEC', 'GCN']

URL = 'https://www.aircanada.com/ca/en/aco/home.html'

In [4]:
def get_to_form(driver):
    driver.get(URL)
    
    # time.sleep(2)
    # WebDriverWait(driver, 5).until(
    #     EC.presence_of_element_located((By.ID, 'bkmgFlights_tripTypeSelector_M'))
    # ).click()

In [5]:
driver = create_driver(HEADLESS)
get_to_form(driver)

In [6]:
form = driver.find_elements(By.TAG_NAME, 'form')[1]

In [7]:
history_table = HistoryTable(
    url=URL,
    xpath=get_xpath(driver, form)
)

In [8]:
for element in form.find_elements(By.TAG_NAME, 'input'):
    try:
        element.clear()
    except:
        pass

# Processing

In [9]:
html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
relation_graph, input_groups = parse_form(
    driver,
    form,
    TEXT_EMBEDDING_METHOD=TEXT_EMBEDDING_METHOD
)

Computing transition probabilities:   0%|          | 0/326 [00:00<?, ?it/s]

100%|██████████████████████████████████████| 32/32 [00:11<00:00,  2.79it/s]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openai/embeddings_utils.py:68: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


# Generation

In [10]:
value_table = generate_constraints_for_input_groups(input_groups)

  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant information section. You must choose your constraints in the format of our modified version of the Jest library in JavaScript. The list of functions in this modified format are:\n1. toBeEqual(value) # the input field value is exactly equal to the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)\n4. toHaveCompareCondit

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_tripTypeSelector_M')).toBeEqual('M')",
        "role": "assistant"
      }
    }
  ],
  "created": 1689635851,
  "id": "chatcmpl-7dRl5XbUWaZ3jTfM7vCIPfaYr41V6",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 675,
    "total_tokens": 692
  }
}
  Total running cost: $0.064
toBeEqual ["'M'"]
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_destination_trip_1'))\n.toBeTruthy()\n.not.toBeEqual(field('bkmgFlights_origin_trip_1'))",
        "role": "assistant"
      }
    }
  ],
  "created": 1689635857,
  "id": "chatcmpl-7dRlBTEOKfj6cJzKSV7gBrjdIc4UC",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 29,
    "prompt_tokens": 653,
    "total_tokens": 682
  }
}
  Total running cost: $0.127
toBeTruthy ['']
toBeEqual ["field('bkmgFlights_origin_trip_1')"]
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from e

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_travelDates_1-formfield-2'))\n.toBeTruthy()\n.toHaveLengthCondition('=', 5)\n.toContainChar('/')\n.toHaveCompareCondition('>', field('bkmgFlights_travelDates_1-formfield-1'))",
        "role": "assistant"
      }
    }
  ],
  "created": 1689635866,
  "id": "chatcmpl-7dRlKSbC5ENVdzZZN6sBQSpLWBXIK",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 53,
    "prompt_tokens": 912,
    "total_tokens": 965
  }
}
  Total running cost: $0.182
toBeTruthy ['']
toHaveLengthCondition ["'='", '5']
toContainChar ["'/'"]
toHaveCompareCondition ["'>'", "field('bkmgFlights_travelDates_1-formfield-1')"]


In [11]:
value_table = generate_values_for_input_groups(input_groups, value_table)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Los Angeles",
        "role": "assistant"
      }
    }
  ],
  "created": 1689635878,
  "id": "chatcmpl-7dRlWR2pfI7XMxWCOt8TF3idZV3la",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 2,
    "prompt_tokens": 292,
    "total_tokens": 294
  }
}
  Total running cost: $0.224
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in ques

In [12]:
fill_form_with_value_table(driver, value_table, input_groups)

<input>value is R</input> at y: (567, 597), x: (219, 319)
R
<input>value is O</input> at y: (567, 597), x: (334, 421)
O
<input>value is M</input> at y: (567, 597), x: (437, 591)
M
<input></input> at y: (574, 594), x: (652, 905)
true
<input></input> at y: (628, 678), x: (198, 466)
Los Angeles
<input></input> at y: (628, 678), x: (536, 804)
New York
<input>DD/MM</input> at y: (628, 678), x: (833, 958)
18/08
<input>DD/MM</input> at y: (628, 678), x: (978, 1093)
20/08


In [13]:
submit_form(driver, input_groups)

0
1
2
3
4


In [14]:
new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

In [15]:
history_table.add(
    value_table.get_values_dict(),
    'base',
    global_feedback,
    driver.current_url
)

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


# Feedback

In [16]:
new_form = driver.find_elements(By.TAG_NAME, 'form')[1]

In [17]:
new_relation_graph, new_input_groups = parse_form(
    driver,
    new_form,
    prev_relation_graph=relation_graph,
    TEXT_EMBEDDING_METHOD=TEXT_EMBEDDING_METHOD
)

Computing transition probabilities:   0%|          | 0/366 [00:00<?, ?it/s]

100%|██████████████████████████████████████| 38/38 [00:11<00:00,  3.41it/s]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openai/embeddings_utils.py:68: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [18]:
global_feedback = get_global_feedback(html, new_html, remove_form_children=True)

In [19]:
global_feedback

['It is not possible to search for flights which have both an origin and a destination in the United States.',
 'It is not possible to book a flight segment between two cities in the same country, unless that country is Canada.  Please contact Air Canada ReservationsOpens in a new tab for assistance with this type of booking.']

In [20]:
value_table = generate_constraints_for_input_groups(
    new_input_groups,
    value_table,
    global_feedback
)

  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant information section. You must choose your constraints in the format of our modified version of the Jest library in JavaScript. The list of functions in this modified format are:\n1. toBeEqual(value) # the input field value is exactly equal to the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)\n4. toHaveCompareCondit

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_tripTypeSelector_M')).toBeEqual('M')",
        "role": "assistant"
      }
    }
  ],
  "created": 1689635952,
  "id": "chatcmpl-7dRmiaUz9IBmburiLjYBBNC2ZSg71",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 833,
    "total_tokens": 850
  }
}
  Total running cost: $0.338
toBeEqual ["'M'"]
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_origin_trip_1'))\n.toBeTruthy()\n.not.toBeEqual(field('bkmgFlights_destination_trip_1'))\n.freeTextConstraint('The origin and destination cannot be in the same country, unless that country is Canada')\n.freeTextConstraint('The origin and destination cannot both be in the United States')",
        "role": "assistant"
      }
    }
  ],
  "created": 1689635956,
  "id": "chatcmpl-7dRmmbxQKPAqnjJU5ECD3EP73Xtl7",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 66,
    "prompt_tokens": 830,
    "total_tokens": 896
  }
}
  Total running cost: $0.388
toBeTruthy ['']
toBeEqual ["field('bkmgFlights_destination_trip_1')"]
freeTextConstraint ["'The origin and destination cannot be in the same country, unless that country is Canada'"]
freeTextConstraint ["'The origin and destination cannot both be in the Un

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_travelDates_1-formfield-1'))\n.toBeTruthy()\n.toHaveLengthCondition('=', 5)\n.toContainChar('/')\n.toHaveCompareCondition('>=', new Date('2023-07-17'))\n.toBeEqual(field('bkmgFlights_travelDates_1-formfield-2'))\n.freeTextConstraint('The date format must be DD/MM')",
        "role": "assistant"
      }
    }
  ],
  "created": 1689635964,
  "id": "chatcmpl-7dRmuVcIEFbuglJdka38Fr3fYT4gc",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 78,
    "prompt_tokens": 941,
    "total_tokens": 1019
  }
}
  Total running cost: $0.448
toBeTruthy ['']
toHaveLengthCondition ["'='", '5']
toContainChar ["'/'"]
toHaveCompareCondition ["'>='", "new Date('2023-07-17')"]
toBeEqual ["field('bkmgFlights_travelDates_1-formfield-2')"]
freeTextConstraint ["'The date format must be DD/MM'"]
  Prompt: [
  {
    "role": "s

In [21]:
value_table = generate_values_for_input_groups(new_input_groups, value_table)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "London",
        "role": "assistant"
      }
    }
  ],
  "created": 1689635980,
  "id": "chatcmpl-7dRnAcxOfrw6oJeAvGhJH89y011MH",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 339,
    "total_tokens": 340
  }
}
  Total running cost: $0.525
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question,

In [22]:
fill_form_with_value_table(driver, value_table, new_input_groups)

<input>value is R</input> at y: (743, 773), x: (219, 319)
R
<input>value is O</input> at y: (743, 773), x: (334, 421)
O
<input>value is M</input> at y: (743, 773), x: (437, 591)
M
<input></input> at y: (750, 770), x: (652, 905)
true
<input></input> at y: (804, 854), x: (198, 466)
London
<input></input> at y: (804, 854), x: (536, 804)
Paris
<input>DD/MM</input> at y: (804, 854), x: (833, 958)
23/08
<input>DD/MM</input> at y: (804, 854), x: (978, 1093)
30/08


In [23]:
submit_form(driver, new_input_groups)

0
1
2
3
4


In [24]:
new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

history_table.add(
    value_table.get_values_dict(),
    'base',
    global_feedback,
    driver.current_url
)

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


# Constraint Negation

In [25]:
form_context = get_form_context(input_groups)

In [26]:
history_table.table = history_table.table[:2]

In [27]:
get_attr = lambda x, a, d: x[a] if a in x else d

for field_id, entry in value_table.entries.items():
    input_type = get_attr(entry.input_group.node.element.attrs, 'type', 'text')
    if input_type in ['radio', 'checkbox']:
        continue
    
    for i in range(len(entry.constraints)):
        get_to_form(driver)
        
        value_table_copy = value_table.copy()
    
        value_table_copy.entries[field_id].constraints = [value_table.entries[field_id].constraints[i].copy()]
        value_table_copy.entries[field_id].constraints[0].flip_negative()


        input_group = value_table_copy.entries[field_id].input_group

        value_table_copy = generate_value_for_input_group(
            input_group,
            value_table_copy,
            form_context=form_context
        )

        fill_form_with_value_table(driver, value_table_copy, input_groups)

        submit_form(driver, input_groups)
        
        time.sleep(1)
        
        new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
        global_feedback = get_global_feedback(html, new_html)
        
        print(global_feedback)
        
        history_table.add(
            value_table_copy.get_values_dict(),
            str(type(value_table_copy.entries[field_id].constraints[0])),
            global_feedback,
            driver.current_url
        )
        
        if len(global_feedback) == 0:
            value_table.entries[field_id].constraints[i].approve()

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

...

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


In [28]:
driver.get(URL)
time.sleep(1)
base_text = driver.find_element(By.TAG_NAME, 'body').text

In [29]:
history_table.table

,values,variation_type,feedback,new_url
0,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",base,It is not possible to search for flights which...,https://www.aircanada.com/ca/en/aco/home.html
1,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",base,"We're sorry, we were not able to process your ...",https://www.aircanada.com/ca/en/aco/home.html
2,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.ToBeT...,Skip to book a flight Skip to main navigation...,https://www.aircanada.com/ca/en/aco/home.html
3,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.ToBe'>,Please select a valid point of origin for this...,https://www.aircanada.com/ca/en/aco/home.html
4,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.FreeT...,Please select a valid point of origin for this...,https://www.aircanada.com/ca/en/aco/home.html
5,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.FreeT...,Please select a valid point of origin for this...,https://www.aircanada.com/ca/en/aco/home.html
6,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.ToBeT...,Skip to book a flight Skip to main navigation...,https://www.aircanada.com/ca/en/aco/home.html
7,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.ToBe'>,The departure and arrival cities/airports you'...,https://www.aircanada.com/ca/en/aco/home.html
8,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.FreeT...,Skip to book a flight Skip to main navigation...,https://www.aircanada.com/ca/en/aco/home.html
9,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.ToBeT...,Please select a valid point of origin for this...,https://www.aircanada.com/ca/en/aco/home.html


In [32]:
history_table.table = history_table.table[
    history_table.table.feedback.str.len() < len(base_text)
]

In [33]:
set('FEEDBACK-DELIMITER'.join(history_table.table.feedback).split('FEEDBACK-DELIMITER'))

{'It is not possible to book a flight segment between two cities in the same country, unless that country is Canada.  Please contact Air Canada ReservationsOpens in a new tab for assistance with this type of booking.',
 'It is not possible to search for flights which have both an origin and a destination in the United States.',
 'Please select a valid destination for this trip.',
 'Please select a valid point of origin for this trip.',
 "The departure and arrival cities/airports you've selected are the same. Please review your selection and try again.",
 "We're sorry, we were not able to process your request. Please try again, or contact usOpens in a new tab for assistance.7",
 'Your return date cannot be before your departure date.'}

In [34]:
len(set('FEEDBACK-DELIMITER'.join(history_table.table.feedback).split('FEEDBACK-DELIMITER')))

7

In [23]:
history_table.to_test_case('aoc-multitrip.py')